<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/EFFICIENTNETPARENTMODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# Average inference time (AIT) for 10 images
# Model: DenseNet121_single_split.keras (parent)
# Prints:
#  - Per-image avg (single-image loop)
#  - Per-image avg (single batched run of 10)
# ============================================

!pip -q install kagglehub

import os, glob, time, random, numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import kagglehub
from google.colab import drive

# --------- Config ---------
MODEL_PATH = "/content/drive/MyDrive/DiabeticProject/saved_models/EfficientNetV2B0_single_split.keras"
N_SAMPLES  = 10
SEED       = 7

# --------- Mount & load model ---------
drive.mount('/content/drive')
assert os.path.exists(MODEL_PATH), f"Model not found: {MODEL_PATH}"

print("Loading model:", MODEL_PATH)
model = keras.models.load_model(MODEL_PATH, compile=False)

# Input size (H, W, 3)
inp_shape = model.input_shape
if isinstance(inp_shape, list):
    inp_shape = inp_shape[0]
H, W = int(inp_shape[1]), int(inp_shape[2])
print(f"Model input size: {H}x{W}x3")

# Optional: jit graph for stable timing
@tf.function(jit_compile=False)
def fwd(x):
    return model(x, training=False)

# --------- Dataset: pick 10 images ---------
ds_path = kagglehub.dataset_download("subhajeetdas/aptos-2019-jpg")
print("Dataset path:", ds_path)

img_paths = sorted(
    glob.glob(os.path.join(ds_path, "**", "*.jpg"), recursive=True) +
    glob.glob(os.path.join(ds_path, "**", "*.png"), recursive=True)
)
if len(img_paths) < N_SAMPLES:
    raise RuntimeError(f"Not enough images found in dataset (found {len(img_paths)}).")

random.seed(SEED)
sample_paths = random.sample(img_paths, N_SAMPLES)
print("Using images:")
for p in sample_paths:
    print(" -", p)

def load_and_preprocess(p):
    im = Image.open(p).convert("RGB").resize((W, H), Image.BILINEAR)
    arr = np.asarray(im, dtype=np.float32) / 255.0  # simple [0,1] scaling for timing
    return arr

# --------- Warm-up ---------
_ = fwd(tf.zeros((1, H, W, 3), dtype=tf.float32))

# --------- Single-image loop timing ---------
per_img_ms = []
for p in sample_paths:
    x = load_and_preprocess(p)
    x = tf.convert_to_tensor(x[None, ...], dtype=tf.float32)  # (1,H,W,3)
    t0 = time.perf_counter()
    y = fwd(x)
    # force materialization
    _ = y if isinstance(y, (list,tuple)) else [y]
    _ = [t.numpy() for t in _]
    t1 = time.perf_counter()
    per_img_ms.append((t1 - t0) * 1000.0)

single_avg = float(np.mean(per_img_ms))

# --------- Batched timing (all 10 at once) ---------
batch = np.stack([load_and_preprocess(p) for p in sample_paths], axis=0)  # (10,H,W,3)
batch = tf.convert_to_tensor(batch, dtype=tf.float32)

t0 = time.perf_counter()
yb = fwd(batch)
_ = yb if isinstance(yb, (list,tuple)) else [yb]
_ = [t.numpy() for t in _]
t1 = time.perf_counter()
batch_total_ms = (t1 - t0) * 1000.0
batch_avg = batch_total_ms / N_SAMPLES

# --------- Print results ---------
print("\n===== AVERAGE INFERENCE TIME (10 images) =====")
print(f"Single-image loop : {single_avg:.3f} ms / image")
print(f"Batched (N={N_SAMPLES}): {batch_avg:.3f} ms / image   (total {batch_total_ms:.3f} ms)")
print("Device:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")


Mounted at /content/drive
Loading model: /content/drive/MyDrive/DiabeticProject/saved_models/EfficientNetV2B0_single_split.keras
Model input size: 256x256x3


100%|██████████| 2.82G/2.82G [00:26<00:00, 112MB/s] 

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12
Using images:
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_02652.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_01265.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_3 (155).png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/No DR/image_01239.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_00388.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_00592.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/No DR/image_02660.png
 - /root/.cache/k